### 使用routechain来实现判断存款还是取款

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain_ollama import OllamaLLM
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

withdraw_template = """你是银行资深柜员，非常善于处理客户取款问题。
你会判断客户到银行做的是什么交易，识别出取款交易给设置当前的 pcode=200101。
你会从问题中识别出客户要取多少钱，比如取3000， 那amount=3000。

这是一个客户业务：
{input}"""


despoit_template =  """你是银行资深柜员，非常善于处理客户存款问题。
你会判断客户到银行做的是什么交易，识别出存款交易给设置当前的 pcode=200102。
你会从问题中识别出客户要存多少钱，比如存3000， 那amount=3000。

这是一个客户业务：
{input}"""

prompt_infos = [
    {
        "name": "存款",
        "description": "适用于回答存款问题",
        "prompt_template": despoit_template,
    },
    {
        "name": "取款",
        "description": "适用于回答取款问题",
        "prompt_template": withdraw_template,
    },
]

In [28]:
llm = OllamaLLM(model="llama3.1:latest",temperature=0.9)


In [29]:
destination_chains = {}
# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [30]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

In [31]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE


In [32]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)


In [33]:
print(destinations)
print(destinations_str)

['存款: 适用于回答存款问题', '取款: 适用于回答取款问题']
存款: 适用于回答存款问题
取款: 适用于回答取款问题


In [34]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [35]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
存款: 适用于回答存款问题
取款: 适用于回答取款问题

<< INPUT >>
{input}

<< OUTPUT (must include ```json at the start of the

In [36]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [37]:
print(chain.invoke("张三到柜面取钱，他需要取5000"))



> Entering new MultiPromptChain chain...
取款: {'input': '张三到柜面取出5000元'}
> Finished chain.
{'input': '张三到柜面取出5000元', 'text': '根据客户的业务，我判断这次客户是来取现金。为了记录该笔交易，我会设置pcode=200202（代表取现交易）。由于客户要取出的是5000元，因此amount=5000。\n\n当前的 pcode = 200202\namount = 5000'}


In [39]:
print(
    chain.invoke(
        "李四到柜面，要存4000元。"
    )
)



> Entering new MultiPromptChain chain...
存款: {'input': '李四到柜面，要求存入4000元'}
> Finished chain.
{'input': '李四到柜面，要求存入4000元', 'text': '由于问题描述了客户 "要求存入" tiền，而不是取款，我判断这是一笔存款交易。所以我需要设置pcode为存款的代码，通常是200102。amount就表示存入的金额，所以将其设为4000。\n\n```python\n# 判断交易类型\nif 问题 == \'存入\':\n    pcode = 200102\nelse:\n    # 不支持其他交易类型\n\n# 设置取款金额\namount = 4000\n```\n\n现在，我已经确定了pcode和amount的值，可以继续处理客户业务。'}


In [13]:
router_chain.verbose = True

In [14]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': 'What is a black hole in terms of astrophysics?'}
> Finished chain.
{'input': 'What is a black hole in terms of astrophysics?', 'text': '黑洞是一种天体物理学中的奇异对象。它是空间中质量极度浓缩的区域，形成自引力的强大场所，以至于连光都无法从其表面逃逸。\n\n你可以想象它像一个有无限密度的物质球，每个粒子都被挤压到了点尺寸，使得它的重力无比之强，从而形成了不允许任何物体逃离的区域。'}
